## Ring Inflation of the Triangle - Compatibility Test
---

In [ ]:
import gurobipy as gp
import numpy as np

In [ ]:
# P_ABC = {(0,0,0): 0,
#          (0,0,1): 1/3,
#          (0,1,0): 1/3,
#          (1,0,0): 1/3,
#          (0,1,1): 0,
#          (1,0,1): 0,
#          (1,1,0): 0,
#          (1,1,1): 0}

P_ABC = {(0,0,0): 1/4,
         (0,0,1): 1/4,
         (0,1,0): 1/4,
         (1,0,0): 1/4,
         (0,1,1): 0,
         (1,0,1): 0,
         (1,1,0): 0,
         (1,1,1): 0}

def get_funcs(P_ABC):
    P_A = np.array([sum([P_ABC[a,b,c] for b,c in np.ndindex(2,2)]) for a in range(2)])
    P_B = np.array([sum([P_ABC[a,b,c] for a,c in np.ndindex(2,2)]) for b in range(2)])
    P_C = np.array([sum([P_ABC[a,b,c] for a,b in np.ndindex(2,2)]) for c in range(2)])

    P_AB = np.array([[sum([P_ABC[a,b,c] for c in range(2)]) for b in range(2)] for a in range(2)])
    P_AC = np.array([[sum([P_ABC[a,b,c] for b in range(2)]) for c in range(2)] for a in range(2)])
    P_BC = np.array([[sum([P_ABC[a,b,c] for a in range(2)]) for c in range(2)] for b in range(2)])

    return P_A, P_B, P_C, P_AB, P_AC, P_BC

P_A, P_B, P_C, P_AB, P_AC, P_BC = get_funcs(P_ABC)

In [ ]:
m = gp.Model()
# m.params.NonConvex = 2

Q_A1_A2_B1_B2_C1_C2 = m.addMVar((2,2,2,2,2, 2), name="P", vtype=gp.GRB.CONTINUOUS, lb=0, ub=1)
m.update()

In [ ]:
def Q_A1_A2_B1_B2(a1,a2,b1,b2):
    return Q_A1_A2_B1_B2_C1_C2[a1,a2,b1,b2,:,:].sum()

def Q_B1_B2_C1_C2(b1,b2,c1,c2):
    return Q_A1_A2_B1_B2_C1_C2[:,:,b1,b2,c1,c2].sum()

def Q_A1_A2_C1_C2(a1,a2,c1,c2):
    return Q_A1_A2_B1_B2_C1_C2[a1,a2,:,:,c1,c2].sum()

def Q_A1_B2_C1(a1,b2,c1):
    return Q_A1_A2_B1_B2_C1_C2[a1,:,:,b2,c1,:].sum()

m.update()

In [ ]:
## Expressability:
for a1,a2,b1,b2 in np.ndindex(2,2,2,2): # (2)
    m.addConstr(Q_A1_A2_B1_B2(a1,a2,b1,b2) == P_AB[a1,b1]*P_AB[a2,b2], 
                name=f"Q_A1_A2_B1_B2({a1},{a2},{b1},{b2}) = {Q_A1_A2_B1_B2(a1,a2,b1,b2)} =  {P_AB[a1,b1]*P_AB[a2,b2]}")

for b1,b2,c1,c2 in np.ndindex(2,2,2,2): # (3)
    m.addConstr(Q_B1_B2_C1_C2(b1,b2,c1,c2) == P_BC[b1,c1]*P_BC[b2,c2], 
                name=f"Q_B1_B2_C1_C2({b1},{b2},{c1},{c2}) = {Q_B1_B2_C1_C2(b1,b2,c1,c2)} = {P_BC[b1,c1]*P_BC[b2,c2]}")

for a1,a2,c1,c2 in np.ndindex(2,2,2,2): # (4)
    m.addConstr(Q_A1_A2_C1_C2(a1,a2,c1,c2) == P_AC[a1,c2]*P_AC[a2,c1], 
                name=f"Q_A1_A2_C1_C2({a1},{a2},{c1},{c2}) = {Q_A1_A2_C1_C2(a1,a2,c1,c2)} =  {P_AC[a1,c2]*P_AC[a2,c1]}")

for a,b,c in np.ndindex(2,2,2): # (5)
    m.addConstr(Q_A1_B2_C1(a,b,c) == P_A[a]*P_B[b]*P_C[c], 
                name=f"Q_A1_B2_C1({a},{b},{c}) = {P_A[a]*P_B[b]*P_C[c]}") # {#Q_A1_B2_C1(a,b,c)} =

# Symmetry:
for a1, a2, b1, b2, c1, c2 in np.ndindex(2, 2, 2, 2, 2, 2): # (6)
    m.addConstr(Q_A1_A2_B1_B2_C1_C2[a1, a2, b1, b2, c1, c2] == Q_A1_A2_B1_B2_C1_C2[a2, a1, b2, b1, c2, c1], 
                name=f"Q_A1_A2_B1_B2_C1_C2({a1},{a2},{b1},{b2},{c1},{c2}) = Q_A1_A2_B1_B2_C1_C2({a2},{a1},{b2},{b1},{c2},{c1})")

# summation to 1:
m.addConstr(Q_A1_A2_B1_B2_C1_C2.sum() == 1)

In [ ]:
## Expressability:
for a1,a2,b1,b2 in np.ndindex(2,2,2,2): # (2)
    print(f"{Q_A1_A2_B1_B2(a1,a2,b1,b2)} =  {P_AB[a1,b1]*P_AB[a2,b2]}")

for b1,b2,c1,c2 in np.ndindex(2,2,2,2): # (3)
    print(f"{Q_B1_B2_C1_C2(b1,b2,c1,c2)} = {P_BC[b1,c1]*P_BC[b2,c2]}")

for a1,a2,c1,c2 in np.ndindex(2,2,2,2): # (4)
    print(f"{Q_A1_A2_C1_C2(a1,a2,c1,c2)} =  {P_AC[a1,c2]*P_AC[a2,c1]}")

for a,b,c in np.ndindex(2,2,2): # (5)
    print(f"{Q_A1_B2_C1(a,b,c)} = {P_A[a]*P_B[b]*P_C[c]}")

# Symmetry:
# for a1, a2, b1, b2, c1, c2 in np.ndindex(2, 2, 2, 2, 2, 2): # (6)
#     print(f"Q_A1_A2_B1_B2_C1_C2({a1},{a2},{b1},{b2},{c1},{c2}) = Q_A1_A2_B1_B2_C1_C2({a2},{a1},{b2},{b1},{c2},{c1})")

In [ ]:
m.optimize()
# check if exists a solution:
if m.status == gp.GRB.OPTIMAL:
    print("Compatible!")
    for a1,a2,b1,b2,c1,c2 in np.ndindex(2,2,2,2,2,2):
        print(f"Q({a1},{a2},{b1},{b2},{c1},{c2}) = {Q_A1_A2_B1_B2_C1_C2[a1,a2,b1,b2,c1,c2].X}")
else:
    print("Incompatible!")

In [ ]:
# Q_shape = (2,2,2,2,2,2)
# n_col = np.prod(Q_shape) # num of columns
# Q_base_old = m.addMVar(shape=Q_shape, vtype=gp.GRB.CONTINUOUS, name="prob(A1,B1,C1,A2,B2,C2)", lb=0, ub=1)
# Q_base = np.eye(64).reshape(Q_shape+(n_col,)) # id matrix



# b_vec = []
# b_vec_symbolic = []
# A_mat = []

# for a1, b1, a2, b2 in np.ndindex(2,2,2,2):
#     A_mat.append(Q_A1_A2_B1_B2(a1,a2,b1,b2))
#     b_vec.append(P_AB[a1,b1]*P_AB[a2,b2])
#     b_vec_symbolic.append(f"P_AB({a1},{b1})*P_AB({a2},{b2})")

# for b1, c1, b2, c2 in np.ndindex(2,2,2,2):
#     A_mat.append(Q_B1_B2_C1_C2(b1,b2,c1,c2))
#     b_vec.append(P_BC[b1,c1]*P_BC[b2,c2])
#     b_vec_symbolic.append(f"P_BC({b1},{c1})*P_BC({b2},{c2})")
# for a1, c1, a2, c2 in np.ndindex(2,2,2,2):
#     A_mat.append(Q_A1_A2_C1_C2(a1,a2,c1,c2))
#     b_vec.append(P_AC[a1,c2]*P_AC[a2,c1])
#     b_vec_symbolic.append(f"P_AC({a1},{c2})*P_AC({a2},{c1})")

# for a1,c1,b2 in np.ndindex(2,2,2):
#     # m.addConstr(Q_A1C1B2(a1,c1,b2) == P_A(a1)*P_C(c1)*P_B(b2))
#     A_mat.append(Q_A1_B2_C1(a1,b2,c1))
#     b_vec.append(P_A[a1]*P_C[c1]*P_B[b2])
#     b_vec_symbolic.append(f"P_A({a1})*P_C({c1})*P_B({b2})")

# A = np.asarray(A_mat)
# b = np.asarray(b_vec)
# # m.addMConstr(A, Q_base_old.reshape(-1), '=', b)
# m.update()